In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-text
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 13.5 MB/s 
     |████████████████████████████████| 588.3 MB 20 kB/s 
     |████████████████████████████████| 439 kB 60.7 MB/s 
     |████████████████████████████████| 6.0 MB 52.1 MB/s 
     |████████████████████████████████| 1.7 MB 43.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_for_colab/IMDB Dataset.csv')

In [ ]:
df['is_positive'] = (df['sentiment'] == 'positive').astype(int)
df.drop(columns=['sentiment'], inplace=True)
df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']

In [ ]:
df

,DATA_COLUMN,LABEL_COLUMN
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [ ]:
df['LABEL_COLUMN'].value_counts()

1    25000
0    25000
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_positive = df[df['LABEL_COLUMN']==1]
df_negative = df[df['LABEL_COLUMN']==0]

In [ ]:
df_negative

,DATA_COLUMN,LABEL_COLUMN
3,Basically there's a family where a little boy ...,0
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
10,Phil the Alien is one of those quirky films wh...,0
11,I saw this movie when I was about 12 when it c...,0
...,...,...
49994,This is your typical junk comedy.<br /><br />T...,0
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [ ]:
# Для тестовой выборки берем последние 10% негативных отзывов и последние 10% позитивных отзывов

In [ ]:
n_test = df_negative.shape[0] // 10 # в оригинале df_negative.shape[0] // 10
df_negative_test = df_negative.tail(n_test)
n_test = df_positive.shape[0] // 10 # df_positive.shape[0] // 10

df_positive_test = df_positive.tail(n_test)

In [ ]:
df_positive_test

,DATA_COLUMN,LABEL_COLUMN
44910,One of the most heart-warming foreign films I'...,1
44911,David Lynch's (1999) film of John Roach / Mary...,1
44916,"This is definitely Nolan's most intimite,and t...",1
44925,Before I watched this tv movie I did not know ...,1
44927,"I just managed to find a copy of ""Mission Cleo...",1
...,...,...
49983,"I loved it, having been a fan of the original ...",1
49985,Imaginary Heroes is clearly the best film of t...,1
49989,I got this one a few weeks ago and love it! It...,1
49992,John Garfield plays a Marine who is blinded by...,1


In [ ]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [ ]:
df_balanced_test['LABEL_COLUMN'].value_counts()

0    2500
1    2500
Name: LABEL_COLUMN, dtype: int64

In [ ]:
# Для обучающей выборки берем первые 2.5% из начала датасета.

In [ ]:
n_train = df_negative.shape[0] // 40 # в оригинале df_negative.shape[0] // 40
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 40 # в оригианале df_positive.shape[0] // 40
df_positive_train = df_positive.head(n_train)

In [ ]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [ ]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['LABEL_COLUMN'].squeeze()

In [ ]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['LABEL_COLUMN'].squeeze()

In [ ]:
labse_preprocess = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")


In [ ]:
labse_encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2")

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = labse_preprocess(text_input)
outputs = labse_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [ ]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [ ]:
history = model.fit(X_train, y_train, epochs=10)

Epoch 1/10
40/40 [==============================] - 564s 14s/step - loss: 0.6735 - accuracy: 0.5864 - precision: 0.5906 - recall: 0.5632
Epoch 2/10
40/40 [==============================] - 553s 14s/step - loss: 0.5942 - accuracy: 0.7008 - precision: 0.7047 - recall: 0.6912
Epoch 3/10
40/40 [==============================] - 550s 14s/step - loss: 0.5374 - accuracy: 0.7640 - precision: 0.7741 - recall: 0.7456
Epoch 4/10
40/40 [==============================] - 547s 14s/step - loss: 0.5086 - accuracy: 0.7784 - precision: 0.7710 - recall: 0.7920
Epoch 5/10
40/40 [==============================] - 553s 14s/step - loss: 0.4877 - accuracy: 0.7952 - precision: 0.8122 - recall: 0.7680
Epoch 6/10
40/40 [==============================] - 549s 14s/step - loss: 0.4722 - accuracy: 0.7960 - precision: 0.8003 - recall: 0.7888
Epoch 7/10
40/40 [==============================] - 548s 14s/step - loss: 0.4577 - accuracy: 0.8128 - precision: 0.8231 - recall: 0.7968
Epoch 8/10
40/40 [=======================

In [ ]:
y_predicted = model.predict(X_test)


157/157 [==============================] - 2166s 14s/step


In [ ]:
y_predicted = y_predicted.flatten()
y_predicted


array([0.28313386, 0.22662157, 0.34900805, ..., 0.9742413 , 0.66199607,
       0.59345865], dtype=float32)

In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:

y_test_array = np.array(y_test)

In [ ]:
accuracy_score(y_test, y_predicted)

0.7706

In [ ]:
precision_score(y_test_array, y_predicted)

0.7451975353388909

In [ ]:
recall_score(y_test_array, y_predicted)

0.8224

In [ ]:
f1_score(y_test_array, y_predicted)

0.7818976991823541

In [ ]:
df_results_for_reviews_on_movies = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [ ]:
df_results_for_reviews_on_movies.loc['reviews on movies', 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_for_reviews_on_movies.loc['reviews on movies', 'precision'] = precision_score(y_test, y_predicted)
df_results_for_reviews_on_movies.loc['reviews on movies', 'recall'] = recall_score(y_test, y_predicted)
df_results_for_reviews_on_movies.loc['reviews on movies', 'f1_score'] =  f1_score(y_test_array, y_predicted)

In [ ]:
df_results_for_reviews_on_movies

,accuracy,precision,recall,f1_score
reviews on movies,0.7706,0.745198,0.8224,0.781898


In [ ]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/labse_trained_on_imdb_28_december'

In [ ]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/labse_trained_on_imdb_28_december


In [ ]:
model.save(saved_model_path, include_optimizer=True) 

In [ ]:
from keras.models import load_model
loaded_model = load_model(saved_model_path)


In [ ]:
y_predicted_loaded = loaded_model.predict(X_test)
y_predicted_loaded = np.where(y_predicted_loaded > 0.5, 1, 0)


157/157 [==============================] - 2199s 14s/step


In [ ]:
df_loaded_results_for_reviews_on_movies = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])
df_loaded_results_for_reviews_on_movies.loc['reviews on movies', 'accuracy'] = accuracy_score(y_test, y_predicted_loaded)
df_loaded_results_for_reviews_on_movies.loc['reviews on movies', 'precision'] = precision_score(y_test, y_predicted_loaded)
df_loaded_results_for_reviews_on_movies.loc['reviews on movies', 'recall'] = recall_score(y_test, y_predicted_loaded)
df_loaded_results_for_reviews_on_movies.loc['reviews on movies', 'f1_score'] =  f1_score(y_test_array, y_predicted_loaded)

In [ ]:
df_loaded_results_for_reviews_on_movies


,accuracy,precision,recall,f1_score
reviews on movies,0.7706,0.745198,0.8224,0.781898
